In [ ]:
"""
WEEK 5: MACHINE LEARNING & REGULARIZATION
Building on Week 4's ARIMA/GARCH forecasts
...
"""

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats  # ← ADD THIS LINE
import matplotlib.pyplot as plt

In [ ]:
# ============================================================================
# CELL 1: LOAD WEEK 4 RESULTS (FORECASTS)
# ============================================================================

def load_week4_forecasts():
    """
    Simulate loading Week 4 ARIMA/GARCH forecast results.
    NOW WITH REALISTIC CORRELATIONS for better ML demonstration.
    """
    print("=" * 70)
    print("📂 LOADING WEEK 4 FORECASTS")
    print("=" * 70)
    
    dates = pd.date_range(end=pd.Timestamp.now(), periods=100, freq='D')
    
    # Create BASE return with some autocorrelation (realistic market behavior)
    np.random.seed(42)  # For reproducibility
    base_returns = np.random.normal(0.001, 0.015, 100)
    
    # Add momentum/trend component
    trend = np.linspace(0, 0.002, 100)
    
    # Create correlated returns (crypto assets tend to move together)
    btc_returns = base_returns + trend + np.random.normal(0, 0.005, 100)
    eth_returns = 0.7 * btc_returns + 0.3 * np.random.normal(0.0015, 0.015, 100)
    doge_returns = 0.5 * btc_returns + 0.5 * np.random.normal(0.0005, 0.02, 100)
    
    # Volatility clustering (GARCH-like behavior)
    btc_vol = np.abs(np.random.normal(0.04, 0.01, 100))
    eth_vol = np.abs(np.random.normal(0.05, 0.015, 100))
    doge_vol = np.abs(np.random.normal(0.06, 0.02, 100))
    
    # Add volatility spillover effects
    btc_vol = btc_vol * (1 + 0.3 * np.abs(btc_returns))
    eth_vol = eth_vol * (1 + 0.3 * np.abs(eth_returns))
    doge_vol = doge_vol * (1 + 0.3 * np.abs(doge_returns))
    
    forecast_data = pd.DataFrame({
        'date': dates,
        'BTC_return_forecast': btc_returns,
        'ETH_return_forecast': eth_returns,
        'DOGE_return_forecast': doge_returns,
        'BTC_volatility_forecast': btc_vol,
        'ETH_volatility_forecast': eth_vol,
        'DOGE_volatility_forecast': doge_vol,
    })
    
    print("✅ Loaded 100 days of forecasts")
    print(f"   Assets: BTC, ETH, DOGE")
    print(f"   Data: Return forecasts + Volatility forecasts (from GARCH)")
    print(f"   📊 Added realistic correlations and volatility clustering")
    return forecast_data

In [ ]:
# ============================================================================
# CELL 2: FEATURE ENGINEERING (Prompt-Assisted)
# ============================================================================
def engineer_portfolio_features(forecast_df):
    """
    Create features for ML model using Week 4 forecasts.
    This demonstrates PROMPT-ASSISTED feature engineering.
    """
    print("\n" + "=" * 70)
    print("🔧 FEATURE ENGINEERING")
    print("=" * 70)
    
    features = forecast_df.copy()
    
    # Feature 1: Risk-Adjusted Returns (Sharpe-like)
    print("\n1. Creating Risk-Adjusted Return features...")
    for asset in ['BTC', 'ETH', 'DOGE']:
        features[f'{asset}_risk_adj_return'] = (
            features[f'{asset}_return_forecast'] / features[f'{asset}_volatility_forecast']
        )
    
    # Feature 2: Momentum indicators (rolling averages)
    print("2. Adding momentum features (7-day rolling avg)...")
    for asset in ['BTC', 'ETH', 'DOGE']:
        features[f'{asset}_momentum'] = (
            features[f'{asset}_return_forecast'].rolling(window=7, min_periods=1).mean()
        )
    
    # Feature 3: Volatility regime (high/low vol indicator)
    print("3. Creating volatility regime features...")
    for asset in ['BTC', 'ETH', 'DOGE']:
        vol_median = features[f'{asset}_volatility_forecast'].median()
        features[f'{asset}_high_vol'] = (
            features[f'{asset}_volatility_forecast'] > vol_median
        ).astype(int)
    
    # Feature 4: Cross-asset features
    print("4. Engineering cross-asset features...")
    features['BTC_ETH_return_spread'] = (
        features['BTC_return_forecast'] - features['ETH_return_forecast']
    )
    features['portfolio_avg_vol'] = features[
        ['BTC_volatility_forecast', 'ETH_volatility_forecast', 'DOGE_volatility_forecast']
    ].mean(axis=1)
    
    print(f"\n✅ Created {len(features.columns) - len(forecast_df.columns)} new features")
    print(f"   Total features: {len(features.columns)}")
    
    return features


In [ ]:
# ============================================================================
# CELL 3: PREPARE DATA FOR ML (Target = Portfolio Return)
# ============================================================================

def prepare_ml_data(features_df):
    """
    Prepare X (features) and y (target) for ML models.
    Target: Predict optimal portfolio return.
    """
    print("\n" + "=" * 70)
    print("📊 PREPARING ML DATASET")
    print("=" * 70)
    
    # Create target variable (next period's portfolio return)
    # Simple equal-weighted portfolio for target
    features_df['target_portfolio_return'] = (
        features_df['BTC_return_forecast'].shift(-1) * 0.33 +
        features_df['ETH_return_forecast'].shift(-1) * 0.33 +
        features_df['DOGE_return_forecast'].shift(-1) * 0.34
    )
    
    # Remove rows with NaN
    features_df = features_df.dropna()
    
    # Select feature columns (exclude date and target)
    feature_cols = [col for col in features_df.columns 
                   if col not in ['date', 'target_portfolio_return']]
    
    X = features_df[feature_cols].values
    y = features_df['target_portfolio_return'].values
    
    print(f"✅ Dataset prepared:")
    print(f"   Samples: {len(X)}")
    print(f"   Features: {X.shape[1]}")
    print(f"   Target: Portfolio return (next period)")
    
    return X, y, feature_cols

In [ ]:
# ============================================================================
# CELL 4: RIDGE REGRESSION (L2 Regularization)
# ============================================================================

def fit_ridge_regression(X, y, alpha_range=np.logspace(-4, 1, 30)):  
    """
    Ridge regression with cross-validation for alpha tuning.
    Ridge penalizes large coefficients: min ||y - Xw||² + α||w||²
    """
    print("\n" + "=" * 70)
    print("🔵 RIDGE REGRESSION (L2 Regularization)")
    print("=" * 70)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Cross-validation to find best alpha
    cv_scores = []
    
    print("\nTuning regularization parameter (alpha)...")
    for alpha in alpha_range:
        ridge = Ridge(alpha=alpha)
        # 5-fold cross-validation
        scores = cross_val_score(ridge, X_scaled, y, cv=5, 
                                scoring='neg_mean_squared_error')
        cv_scores.append(-scores.mean())  # Convert to positive MSE
    
    # Find best alpha
    best_idx = np.argmin(cv_scores)
    best_alpha = alpha_range[best_idx]
    
    print(f"✅ Best alpha: {best_alpha:.4f}")
    print(f"   CV MSE: {cv_scores[best_idx]:.6f}")
    
    # Fit final model with best alpha
    best_ridge = Ridge(alpha=best_alpha)
    best_ridge.fit(X_scaled, y)
    
    return best_ridge, scaler, best_alpha, cv_scores

In [ ]:
# ============================================================================
# CELL 5: LASSO REGRESSION (L1 Regularization - Feature Selection)
# ============================================================================

def fit_lasso_regression(X, y, alpha_range=np.logspace(-6, -1, 40)): 
    """
    Lasso regression for automatic feature selection.
    Lasso creates sparse solutions: min ||y - Xw||² + α||w||₁
    """
    print("\n" + "=" * 70)
    print("🟢 LASSO REGRESSION (L1 Regularization)")
    print("=" * 70)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Cross-validation to find best alpha
    cv_scores = []
    n_features_used = []
    
    print("\nTuning alpha and performing feature selection...")
    for alpha in alpha_range:
        lasso = Lasso(alpha=alpha, max_iter=10000)
        scores = cross_val_score(lasso, X_scaled, y, cv=5, 
                                scoring='neg_mean_squared_error')
        cv_scores.append(-scores.mean())
        
        # Fit to count non-zero features
        lasso.fit(X_scaled, y)
        n_features_used.append(np.sum(lasso.coef_ != 0))
    
    # Find best alpha
    best_idx = np.argmin(cv_scores)
    best_alpha = alpha_range[best_idx]
    
    print(f"✅ Best alpha: {best_alpha:.6f}")
    print(f"   CV MSE: {cv_scores[best_idx]:.6f}")
    print(f"   Features selected: {n_features_used[best_idx]}/{X.shape[1]}")
    
    # Fit final model
    best_lasso = Lasso(alpha=best_alpha, max_iter=10000)
    best_lasso.fit(X_scaled, y)
    
    return best_lasso, scaler, best_alpha, n_features_used[best_idx]



In [ ]:
# ============================================================================
# CELL 5A: K-FOLD CROSS-VALIDATION ON OPTIMIZED MODELS
# ============================================================================

def validate_optimized_models_kfold(X, y, best_ridge_alpha, best_lasso_alpha, n_splits=5):
    """
    Apply K-Fold Cross-Validation to the OPTIMIZED Ridge and Lasso models.
    This validates that the hyperparameter-tuned models generalize well.
    
    Parameters:
    -----------
    X : array-like
        Feature matrix
    y : array-like
        Target variable
    best_ridge_alpha : float
        Optimal alpha from Ridge hyperparameter tuning (Cell 4)
    best_lasso_alpha : float
        Optimal alpha from Lasso hyperparameter tuning (Cell 5)
    n_splits : int
        Number of K-Fold splits (default=5)
    
    Returns:
    --------
    dict : Validation results for both models
    """
    print("\n" + "=" * 70)
    print("🔀 K-FOLD VALIDATION OF OPTIMIZED MODELS")
    print("=" * 70)
    
    print(f"\n📚 Validating Hyperparameter-Tuned Models:")
    print(f"   • Ridge alpha: {best_ridge_alpha:.4f} (from Cell 4)")
    print(f"   • Lasso alpha: {best_lasso_alpha:.6f} (from Cell 5)")
    print(f"   • K-Fold splits: {n_splits}")
    print(f"   • Purpose: Verify models generalize to unseen data\n")
    
    # Initialize K-Fold
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Store results for each fold
    ridge_results = []
    lasso_results = []
    
    print(f"{'Fold':<6} {'Size':<12} {'Ridge Train':<15} {'Ridge Test':<15} {'Lasso Train':<15} {'Lasso Test':<15} {'Lasso Features':<15}")
    print("-" * 105)
    
    # Iterate through each fold
    for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(X_scaled), 1):
        # Split data
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Train Ridge with OPTIMIZED alpha
        ridge_model = Ridge(alpha=best_ridge_alpha)
        ridge_model.fit(X_train, y_train)
        ridge_train_pred = ridge_model.predict(X_train)
        ridge_test_pred = ridge_model.predict(X_test)
        ridge_train_mse = mean_squared_error(y_train, ridge_train_pred)
        ridge_test_mse = mean_squared_error(y_test, ridge_test_pred)
        ridge_train_r2 = r2_score(y_train, ridge_train_pred)
        ridge_test_r2 = r2_score(y_test, ridge_test_pred)
        
        # Train Lasso with OPTIMIZED alpha
        lasso_model = Lasso(alpha=best_lasso_alpha, max_iter=10000)
        lasso_model.fit(X_train, y_train)
        lasso_train_pred = lasso_model.predict(X_train)
        lasso_test_pred = lasso_model.predict(X_test)
        lasso_train_mse = mean_squared_error(y_train, lasso_train_pred)
        lasso_test_mse = mean_squared_error(y_test, lasso_test_pred)
        lasso_train_r2 = r2_score(y_train, lasso_train_pred)
        lasso_test_r2 = r2_score(y_test, lasso_test_pred)
        
        # Count non-zero features in Lasso
        n_features_lasso = np.sum(np.abs(lasso_model.coef_) > 1e-5)
        
        ridge_results.append({
            'fold': fold_idx,
            'train_size': len(train_idx),
            'test_size': len(test_idx),
            'train_mse': ridge_train_mse,
            'test_mse': ridge_test_mse,
            'train_r2': ridge_train_r2,
            'test_r2': ridge_test_r2
        })
        
        lasso_results.append({
            'fold': fold_idx,
            'train_size': len(train_idx),
            'test_size': len(test_idx),
            'train_mse': lasso_train_mse,
            'test_mse': lasso_test_mse,
            'train_r2': lasso_train_r2,
            'test_r2': lasso_test_r2,
            'n_features': n_features_lasso
        })
        
        print(f"Fold {fold_idx:<2} {len(train_idx)}/{len(test_idx):<8} "
              f"{ridge_train_mse:<15.6f} {ridge_test_mse:<15.6f} "
              f"{lasso_train_mse:<15.6f} {lasso_test_mse:<15.6f} "
              f"{n_features_lasso:<15}")
    
    print("-" * 105)
    
    # Calculate statistics for Ridge
    ridge_avg_train_mse = np.mean([r['train_mse'] for r in ridge_results])
    ridge_avg_test_mse = np.mean([r['test_mse'] for r in ridge_results])
    ridge_std_test_mse = np.std([r['test_mse'] for r in ridge_results])
    ridge_avg_test_r2 = np.mean([r['test_r2'] for r in ridge_results])
    
    # Calculate statistics for Lasso
    lasso_avg_train_mse = np.mean([r['train_mse'] for r in lasso_results])
    lasso_avg_test_mse = np.mean([r['test_mse'] for r in lasso_results])
    lasso_std_test_mse = np.std([r['test_mse'] for r in lasso_results])
    lasso_avg_test_r2 = np.mean([r['test_r2'] for r in lasso_results])
    lasso_avg_features = np.mean([r['n_features'] for r in lasso_results])
    lasso_min_features = min([r['n_features'] for r in lasso_results])
    lasso_max_features = max([r['n_features'] for r in lasso_results])
    
    print(f"{'AVERAGE':<6} {'':<12} "
          f"{ridge_avg_train_mse:<15.6f} {ridge_avg_test_mse:<15.6f} "
          f"{lasso_avg_train_mse:<15.6f} {lasso_avg_test_mse:<15.6f} "
          f"{lasso_avg_features:<15.1f}")
    print(f"{'STD DEV':<6} {'':<12} "
          f"{'':<15} {ridge_std_test_mse:<15.6f} "
          f"{'':<15} {lasso_std_test_mse:<15.6f} "
          f"{'':<15}")
    
    # Detailed interpretation
    print("\n" + "=" * 70)
    print("📊 VALIDATION RESULTS")
    print("=" * 70)
    
    print(f"\n🔵 RIDGE (L2) - Alpha={best_ridge_alpha:.4f}:")
    print(f"   • Average Test MSE: {ridge_avg_test_mse:.6f} (±{ridge_std_test_mse:.6f})")
    print(f"   • Average Test R²: {ridge_avg_test_r2:.4f}")
    print(f"   • Train/Test Gap: {abs(ridge_avg_train_mse - ridge_avg_test_mse):.6f}")
    
    if ridge_avg_train_mse < ridge_avg_test_mse * 0.75:
        print("   ⚠️  WARNING: Significant overfitting detected")
    elif abs(ridge_avg_train_mse - ridge_avg_test_mse) / ridge_avg_test_mse < 0.15:
        print("   ✅ GOOD: Model generalizes well to unseen data")
    else:
        print("   🔄 MODERATE: Acceptable generalization")
    
    cv_ridge = ridge_std_test_mse / ridge_avg_test_mse
    print(f"   • Coefficient of Variation: {cv_ridge*100:.2f}%", end="")
    if cv_ridge < 0.15:
        print(" ✅ (Very stable across folds)")
    elif cv_ridge < 0.30:
        print(" 🔄 (Reasonably stable)")
    else:
        print(" ⚠️  (High variability)")
    
    print(f"\n🟢 LASSO (L1) - Alpha={best_lasso_alpha:.6f}:")
    print(f"   • Average Test MSE: {lasso_avg_test_mse:.6f} (±{lasso_std_test_mse:.6f})")
    print(f"   • Average Test R²: {lasso_avg_test_r2:.4f}")
    print(f"   • Train/Test Gap: {abs(lasso_avg_train_mse - lasso_avg_test_mse):.6f}")
    
    if lasso_avg_train_mse < lasso_avg_test_mse * 0.75:
        print("   ⚠️  WARNING: Significant overfitting detected")
    elif abs(lasso_avg_train_mse - lasso_avg_test_mse) / lasso_avg_test_mse < 0.15:
        print("   ✅ GOOD: Model generalizes well to unseen data")
    else:
        print("   🔄 MODERATE: Acceptable generalization")
    
    cv_lasso = lasso_std_test_mse / lasso_avg_test_mse
    print(f"   • Coefficient of Variation: {cv_lasso*100:.2f}%", end="")
    if cv_lasso < 0.15:
        print(" ✅ (Very stable across folds)")
    elif cv_lasso < 0.30:
        print(" 🔄 (Reasonably stable)")
    else:
        print(" ⚠️  (High variability)")
    
    print(f"   • Feature Selection: {lasso_avg_features:.1f}/{X.shape[1]} features " +
          f"(range: {lasso_min_features}-{lasso_max_features})")
    
    # Determine winner
    print("\n" + "=" * 70)
    print("🏆 MODEL COMPARISON")
    print("=" * 70)
    
    if ridge_avg_test_mse < lasso_avg_test_mse:
        diff_pct = (lasso_avg_test_mse - ridge_avg_test_mse) / lasso_avg_test_mse * 100
        print(f"\n✅ WINNER: Ridge (L2 Regularization)")
        print(f"   • {diff_pct:.2f}% lower test MSE than Lasso")
        print(f"   • Uses all {X.shape[1]} features with coefficient shrinkage")
        print(f"   • Better when all features contain signal")
    else:
        diff_pct = (ridge_avg_test_mse - lasso_avg_test_mse) / ridge_avg_test_mse * 100
        print(f"\n✅ WINNER: Lasso (L1 Regularization)")
        print(f"   • {diff_pct:.2f}% lower test MSE than Ridge")
        print(f"   • Achieves sparsity with only {lasso_avg_features:.1f}/{X.shape[1]} features")
        print(f"   • Better when many features are irrelevant")
    
    # Statistical significance check
    from scipy import stats
    ridge_test_scores = [r['test_mse'] for r in ridge_results]
    lasso_test_scores = [r['test_mse'] for r in lasso_results]
    t_stat, p_value = stats.ttest_rel(ridge_test_scores, lasso_test_scores)
    
    print(f"\n📉 Statistical Test (Paired t-test):")
    print(f"   • p-value: {p_value:.4f}")
    if p_value < 0.05:
        print(f"   • Result: Difference is statistically significant (p < 0.05)")
    else:
        print(f"   • Result: No significant difference (p ≥ 0.05)")
    
    # Visualize results
    visualize_validation_results(ridge_results, lasso_results, X.shape[1], 
                                best_ridge_alpha, best_lasso_alpha)
    
    # Return validation summary
    validation_summary = {
        'ridge': {
            'alpha': best_ridge_alpha,
            'avg_test_mse': ridge_avg_test_mse,
            'std_test_mse': ridge_std_test_mse,
            'avg_test_r2': ridge_avg_test_r2,
            'cv': cv_ridge
        },
        'lasso': {
            'alpha': best_lasso_alpha,
            'avg_test_mse': lasso_avg_test_mse,
            'std_test_mse': lasso_std_test_mse,
            'avg_test_r2': lasso_avg_test_r2,
            'cv': cv_lasso,
            'avg_features': lasso_avg_features
        },
        'winner': 'ridge' if ridge_avg_test_mse < lasso_avg_test_mse else 'lasso',
        'p_value': p_value
    }
    
    return validation_summary, ridge_results, lasso_results


def visualize_validation_results(ridge_results, lasso_results, total_features, 
                                 ridge_alpha, lasso_alpha):
    """
    Create comprehensive visualization of K-Fold validation results.
    """
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)
    
    folds = [r['fold'] for r in ridge_results]
    
    # Extract metrics
    ridge_train_mse = [r['train_mse'] for r in ridge_results]
    ridge_test_mse = [r['test_mse'] for r in ridge_results]
    ridge_test_r2 = [r['test_r2'] for r in ridge_results]
    
    lasso_train_mse = [r['train_mse'] for r in lasso_results]
    lasso_test_mse = [r['test_mse'] for r in lasso_results]
    lasso_test_r2 = [r['test_r2'] for r in lasso_results]
    lasso_n_features = [r['n_features'] for r in lasso_results]
    
    # Plot 1: Ridge Train vs Test MSE
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(folds, ridge_train_mse, marker='o', label='Train MSE', 
             linewidth=2.5, markersize=9, color='#2E86AB')
    ax1.plot(folds, ridge_test_mse, marker='s', label='Test MSE', 
             linewidth=2.5, markersize=9, color='#A23B72')
    ax1.axhline(np.mean(ridge_test_mse), color='#A23B72', linestyle='--', alpha=0.6, linewidth=2)
    ax1.fill_between(folds, ridge_test_mse, alpha=0.2, color='#A23B72')
    ax1.set_xlabel('Fold Number', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Mean Squared Error', fontsize=11, fontweight='bold')
    ax1.set_title(f'🔵 Ridge (α={ridge_alpha:.4f}): Train vs Test', 
                  fontsize=12, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3, linestyle='--')
    
    # Plot 2: Lasso Train vs Test MSE
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(folds, lasso_train_mse, marker='o', label='Train MSE', 
             linewidth=2.5, markersize=9, color='#06A77D')
    ax2.plot(folds, lasso_test_mse, marker='s', label='Test MSE', 
             linewidth=2.5, markersize=9, color='#D64933')
    ax2.axhline(np.mean(lasso_test_mse), color='#D64933', linestyle='--', alpha=0.6, linewidth=2)
    ax2.fill_between(folds, lasso_test_mse, alpha=0.2, color='#D64933')
    ax2.set_xlabel('Fold Number', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Mean Squared Error', fontsize=11, fontweight='bold')
    ax2.set_title(f'🟢 Lasso (α={lasso_alpha:.6f}): Train vs Test', 
                  fontsize=12, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3, linestyle='--')
    
    # Plot 3: Test MSE Comparison
    ax3 = fig.add_subplot(gs[0, 2])
    x = np.arange(len(folds))
    width = 0.35
    ax3.bar(x - width/2, ridge_test_mse, width, label='Ridge', 
            color='#2E86AB', alpha=0.8, edgecolor='black', linewidth=1.2)
    ax3.bar(x + width/2, lasso_test_mse, width, label='Lasso', 
            color='#06A77D', alpha=0.8, edgecolor='black', linewidth=1.2)
    ax3.set_xlabel('Fold Number', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Test MSE', fontsize=11, fontweight='bold')
    ax3.set_title('Test MSE: Ridge vs Lasso', fontsize=12, fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(folds)
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, axis='y', linestyle='--')
    
    # Plot 4: R² Scores Comparison
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.plot(folds, ridge_test_r2, marker='o', label='Ridge R²', 
             linewidth=2.5, markersize=9, color='#2E86AB')
    ax4.plot(folds, lasso_test_r2, marker='s', label='Lasso R²', 
             linewidth=2.5, markersize=9, color='#06A77D')
    ax4.axhline(0, color='red', linestyle='--', alpha=0.5, linewidth=1.5)
    ax4.set_xlabel('Fold Number', fontsize=11, fontweight='bold')
    ax4.set_ylabel('R² Score', fontsize=11, fontweight='bold')
    ax4.set_title('Model Performance: R² Scores', fontsize=12, fontweight='bold')
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3, linestyle='--')
    
    # Plot 5: Box Plot Comparison
    ax5 = fig.add_subplot(gs[1, 1])
    box_data = [ridge_test_mse, lasso_test_mse]
    bp = ax5.boxplot(box_data, labels=['Ridge', 'Lasso'], patch_artist=True,
                     boxprops=dict(linewidth=2), whiskerprops=dict(linewidth=2),
                     capprops=dict(linewidth=2), medianprops=dict(linewidth=2.5, color='red'))
    bp['boxes'][0].set_facecolor('#2E86AB')
    bp['boxes'][0].set_alpha(0.6)
    bp['boxes'][1].set_facecolor('#06A77D')
    bp['boxes'][1].set_alpha(0.6)
    ax5.set_ylabel('Test MSE', fontsize=11, fontweight='bold')
    ax5.set_title('Test MSE Distribution', fontsize=12, fontweight='bold')
    ax5.grid(True, alpha=0.3, axis='y', linestyle='--')
    
    # Plot 6: Lasso Feature Selection
    ax6 = fig.add_subplot(gs[1, 2])
    bars = ax6.bar(folds, lasso_n_features, color='#06A77D', alpha=0.8, 
                   edgecolor='black', linewidth=1.5)
    ax6.axhline(total_features, color='red', linestyle='--', 
                label=f'Total ({total_features})', linewidth=2.5)
    ax6.axhline(np.mean(lasso_n_features), color='blue', linestyle='--', 
                label=f'Avg ({np.mean(lasso_n_features):.1f})', linewidth=2.5)
    
    # Add value labels on bars
    for bar, val in zip(bars, lasso_n_features):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(val)}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    ax6.set_xlabel('Fold Number', fontsize=11, fontweight='bold')
    ax6.set_ylabel('Number of Features', fontsize=11, fontweight='bold')
    ax6.set_title('🟢 Lasso: Feature Selection', fontsize=12, fontweight='bold')
    ax6.legend(fontsize=10)
    ax6.grid(True, alpha=0.3, axis='y', linestyle='--')
    
    # Plot 7: Overfitting Analysis (Train-Test Gap)
    ax7 = fig.add_subplot(gs[2, 0])
    ridge_gaps = [train - test for train, test in zip(ridge_train_mse, ridge_test_mse)]
    lasso_gaps = [train - test for train, test in zip(lasso_train_mse, lasso_test_mse)]
    
    x = np.arange(len(folds))
    width = 0.35
    ax7.bar(x - width/2, ridge_gaps, width, label='Ridge Gap', 
            color='#2E86AB', alpha=0.7, edgecolor='black')
    ax7.bar(x + width/2, lasso_gaps, width, label='Lasso Gap', 
            color='#06A77D', alpha=0.7, edgecolor='black')
    ax7.axhline(0, color='black', linestyle='-', linewidth=1)
    ax7.set_xlabel('Fold Number', fontsize=11, fontweight='bold')
    ax7.set_ylabel('Train MSE - Test MSE', fontsize=11, fontweight='bold')
    ax7.set_title('Overfitting Analysis (Train-Test Gap)', fontsize=12, fontweight='bold')
    ax7.set_xticks(x)
    ax7.set_xticklabels(folds)
    ax7.legend(fontsize=10)
    ax7.grid(True, alpha=0.3, axis='y', linestyle='--')
    
    # Plot 8: Stability Analysis (CV)
    ax8 = fig.add_subplot(gs[2, 1])
    ridge_cv = np.std(ridge_test_mse) / np.mean(ridge_test_mse) * 100
    lasso_cv = np.std(lasso_test_mse) / np.mean(lasso_test_mse) * 100
    
    models = ['Ridge', 'Lasso']
    cvs = [ridge_cv, lasso_cv]
    colors = ['#2E86AB', '#06A77D']
    bars = ax8.bar(models, cvs, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
    
    # Add value labels
    for bar, val in zip(bars, cvs):
        height = bar.get_height()
        ax8.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.2f}%', ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    ax8.axhline(15, color='orange', linestyle='--', label='Threshold (15%)', linewidth=2)
    ax8.set_ylabel('Coefficient of Variation (%)', fontsize=11, fontweight='bold')
    ax8.set_title('Model Stability (Lower = Better)', fontsize=12, fontweight='bold')
    ax8.legend(fontsize=10)
    ax8.grid(True, alpha=0.3, axis='y', linestyle='--')
    
    # Plot 9: Summary Statistics
    ax9 = fig.add_subplot(gs[2, 2])
    ax9.axis('off')
    
    summary_text = f"""
    📊 VALIDATION SUMMARY
    
    Ridge (L2):
    • Alpha: {ridge_alpha:.4f}
    • Avg Test MSE: {np.mean(ridge_test_mse):.6f}
    • Std Dev: {np.std(ridge_test_mse):.6f}
    • Avg R²: {np.mean(ridge_test_r2):.4f}
    • CV: {ridge_cv:.2f}%
    • Features: All ({total_features})
    
    Lasso (L1):
    • Alpha: {lasso_alpha:.6f}
    • Avg Test MSE: {np.mean(lasso_test_mse):.6f}
    • Std Dev: {np.std(lasso_test_mse):.6f}
    • Avg R²: {np.mean(lasso_test_r2):.4f}
    • CV: {lasso_cv:.2f}%
    • Features: {np.mean(lasso_n_features):.1f}/{total_features}
    
    🏆 Winner: {"Ridge" if np.mean(ridge_test_mse) < np.mean(lasso_test_mse) else "Lasso"}
    Δ MSE: {abs(np.mean(ridge_test_mse) - np.mean(lasso_test_mse)):.6f}
    """
    
    ax9.text(0.05, 0.5, summary_text, fontsize=10, verticalalignment='center',
             fontfamily='monospace', 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4, pad=1))
    
    plt.suptitle('K-Fold Cross-Validation: Comprehensive Model Validation', 
                 fontsize=14, fontweight='bold', y=0.998)
    plt.show()
    
    print("\n🎯 Key Validation Insights:")
    print("   ✅ Optimized hyperparameters validated across multiple folds")
    print("   ✅ Train-test gaps indicate generalization capability")
    print("   ✅ Low CV% shows model stability and robustness")
    print("   ✅ R² scores confirm predictive power")
    print("   ✅ Lasso feature selection is consistent across folds\n")

In [ ]:
# ============================================================================
# CELL 6: COMPARE MODELS & FEATURE IMPORTANCE
# ============================================================================

def compare_models(ridge_model, lasso_model, X, y, feature_names):
    """
    Compare Ridge vs Lasso performance and feature importance.
    """
    print("\n" + "=" * 70)
    print("📈 MODEL COMPARISON")
    print("=" * 70)
    
    # Standardize for both models
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Predictions
    ridge_pred = ridge_model.predict(X_scaled)
    lasso_pred = lasso_model.predict(X_scaled)
    
    # Metrics
    ridge_r2 = r2_score(y, ridge_pred)
    lasso_r2 = r2_score(y, lasso_pred)
    ridge_mse = mean_squared_error(y, ridge_pred)
    lasso_mse = mean_squared_error(y, lasso_pred)
    
    print("\n📊 Performance Metrics:")
    print(f"{'Model':<15} {'R²':<10} {'MSE':<12} {'Features':<10}")
    print("-" * 50)
    print(f"{'Ridge':<15} {ridge_r2:<10.4f} {ridge_mse:<12.6f} {'All':<10}")
    print(f"{'Lasso':<15} {lasso_r2:<10.4f} {lasso_mse:<12.6f} {np.sum(lasso_model.coef_ != 0):<10}")
    
    # Feature importance (Lasso)
    print("\n🎯 Top 5 Important Features (Lasso):")
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': np.abs(lasso_model.coef_)
    }).sort_values('coefficient', ascending=False)
    
    for i, row in coef_df.head(5).iterrows():
        if row['coefficient'] > 0:
            print(f"   {row['feature']:<30} {row['coefficient']:.4f}")
    
    return ridge_pred, lasso_pred

In [ ]:
# ============================================================================
# CELL 7: PORTFOLIO OPTIMIZATION WITH REGULARIZATION
# ===========================================================================
def optimize_portfolio_weights(features_df, lasso_model, scaler):
    """
    Use Lasso predictions to optimize portfolio weights.
    NOW: Uses rolling average + volatility penalty for diversification.
    Apply constraints: weights sum to 1, no short selling.
    """
    print("\n" + "=" * 70)
    print("💼 PORTFOLIO WEIGHT OPTIMIZATION")
    print("=" * 70)
    
    # Use ROLLING AVERAGE of last 30 days (not just last row)
    lookback_period = min(30, len(features_df))
    recent_data = features_df.tail(lookback_period)
    
    # Calculate average risk-adjusted returns
    risk_adj_returns = {
        'BTC': recent_data['BTC_risk_adj_return'].mean(),
        'ETH': recent_data['ETH_risk_adj_return'].mean(),
        'DOGE': recent_data['DOGE_risk_adj_return'].mean()
    }
    
    # Calculate average volatilities
    avg_volatility = {
        'BTC': recent_data['BTC_volatility_forecast'].mean(),
        'ETH': recent_data['ETH_volatility_forecast'].mean(),
        'DOGE': recent_data['DOGE_volatility_forecast'].mean()
    }
    
    # Sharpe-like score: Higher return/vol is better
    sharpe_scores = {
        asset: risk_adj_returns[asset] / (avg_volatility[asset] + 1e-8)
        for asset in ['BTC', 'ETH', 'DOGE']
    }
    
    # Convert to positive scores (shift if negative)
    min_score = min(sharpe_scores.values())
    if min_score < 0:
        sharpe_scores = {k: v - min_score + 0.1 for k, v in sharpe_scores.items()}
    
    # Apply softmax with temperature parameter for diversification
    temperature = 2.0  # Higher = more diversified (try 0.5 to 5.0)
    exp_scores = {k: np.exp(v / temperature) for k, v in sharpe_scores.items()}
    total = sum(exp_scores.values())
    optimal_weights = {k: v / total for k, v in exp_scores.items()}
    
    # Diversification penalty: Reduce concentration
    # If any weight > 50%, redistribute
    max_weight = max(optimal_weights.values())
    if max_weight > 0.5:
        # Apply diversification constraint
        for asset in optimal_weights:
            if optimal_weights[asset] > 0.5:
                excess = optimal_weights[asset] - 0.5
                optimal_weights[asset] = 0.5
                # Redistribute excess to others proportionally
                other_assets = [a for a in optimal_weights if a != asset]
                for other in other_assets:
                    optimal_weights[other] += excess / len(other_assets)
    
    # Ensure weights sum to 1.0
    total_weight = sum(optimal_weights.values())
    optimal_weights = {k: v / total_weight for k, v in optimal_weights.items()}
    
    # Calculate expected portfolio metrics
    expected_return = sum(
        optimal_weights[asset] * risk_adj_returns[asset] 
        for asset in optimal_weights
    )
    
    expected_vol = sum(
        optimal_weights[asset] * avg_volatility[asset] 
        for asset in optimal_weights
    )
    
    portfolio_sharpe = expected_return / expected_vol if expected_vol > 0 else 0
    
    print(f"\n📊 Optimal Portfolio Allocation (Last {lookback_period} days):")
    print(f"{'Asset':<10} {'Weight':<12} {'Avg Risk-Adj Ret':<20} {'Avg Vol':<12} {'Sharpe':<10}")
    print("-" * 75)
    for asset in ['BTC', 'ETH', 'DOGE']:
        print(f"{asset:<10} {optimal_weights[asset]*100:>6.2f}%     "
              f"{risk_adj_returns[asset]:>10.4f}          "
              f"{avg_volatility[asset]:>8.4f}      "
              f"{sharpe_scores[asset]:>8.4f}")
    
    print("\n" + "-" * 75)
    print(f"📈 Expected Portfolio Return:  {expected_return:>10.4f}")
    print(f"📉 Expected Portfolio Vol:     {expected_vol:>10.4f}")
    print(f"⭐ Portfolio Sharpe Ratio:     {portfolio_sharpe:>10.4f}")
    print(f"🎯 Diversification Score:      {1 - max(optimal_weights.values()):.2%}")
    
    return optimal_weights

In [ ]:
# ============================================================================
# CELL 7A: PORTFOLIO VISUALIZATION & ANALYSIS
# ============================================================================

def visualize_portfolio_analysis(optimal_weights, features_df, lasso_model, 
                                 feature_names, validation_summary):
    """
    Create comprehensive portfolio and model analysis visualizations.
    """
    import matplotlib.patches as mpatches
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.35, wspace=0.35)
    
    # ========================================================================
    # PLOT 1: Portfolio Allocation Pie Chart
    # ========================================================================
    ax1 = fig.add_subplot(gs[0, 0])
    
    colors = ['#2E86AB', '#A23B72', '#F18F01']
    explode = (0.05, 0.05, 0.05)
    
    wedges, texts, autotexts = ax1.pie(
        optimal_weights.values(),
        labels=optimal_weights.keys(),
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        explode=explode,
        shadow=True,
        textprops={'fontsize': 12, 'fontweight': 'bold'}
    )
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontsize(14)
    
    ax1.set_title('💼 Optimal Portfolio Allocation', 
                  fontsize=14, fontweight='bold', pad=20)
    
    # Add diversification score
    div_score = 1 - max(optimal_weights.values())
    ax1.text(0, -1.4, f'Diversification Score: {div_score:.1%}', 
             ha='center', fontsize=11, style='italic',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # ========================================================================
    # PLOT 2: Asset Weights Bar Chart with Metrics
    # ========================================================================
    ax2 = fig.add_subplot(gs[0, 1])
    
    lookback = min(30, len(features_df))
    recent_data = features_df.tail(lookback)
    
    metrics = {
        'BTC': {
            'weight': optimal_weights['BTC'],
            'return': recent_data['BTC_risk_adj_return'].mean(),
            'vol': recent_data['BTC_volatility_forecast'].mean()
        },
        'ETH': {
            'weight': optimal_weights['ETH'],
            'return': recent_data['ETH_risk_adj_return'].mean(),
            'vol': recent_data['ETH_volatility_forecast'].mean()
        },
        'DOGE': {
            'weight': optimal_weights['DOGE'],
            'return': recent_data['DOGE_risk_adj_return'].mean(),
            'vol': recent_data['DOGE_volatility_forecast'].mean()
        }
    }
    
    assets = list(metrics.keys())
    weights = [m['weight'] * 100 for m in metrics.values()]
    
    bars = ax2.barh(assets, weights, color=colors, alpha=0.8, 
                    edgecolor='black', linewidth=2)
    
    # Add value labels
    for bar, weight in zip(bars, weights):
        ax2.text(weight + 1, bar.get_y() + bar.get_height()/2, 
                f'{weight:.1f}%',
                va='center', fontweight='bold', fontsize=11)
    
    ax2.set_xlabel('Portfolio Weight (%)', fontsize=11, fontweight='bold')
    ax2.set_title('📊 Asset Allocation Breakdown', 
                  fontsize=14, fontweight='bold')
    ax2.set_xlim(0, max(weights) * 1.15)
    ax2.grid(axis='x', alpha=0.3, linestyle='--')
    
    # ========================================================================
    # PLOT 3: Risk-Return Scatter
    # ========================================================================
    ax3 = fig.add_subplot(gs[0, 2])
    
    for i, (asset, data) in enumerate(metrics.items()):
        ax3.scatter(data['vol']*100, data['return']*100, 
                   s=data['weight']*3000,  # Size by weight
                   c=[colors[i]], alpha=0.7, 
                   edgecolors='black', linewidth=2,
                   label=f"{asset} ({data['weight']*100:.1f}%)")
    
    ax3.set_xlabel('Volatility (%)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Risk-Adjusted Return (%)', fontsize=11, fontweight='bold')
    ax3.set_title('📈 Risk-Return Profile\n(Bubble size = Portfolio weight)', 
                  fontsize=14, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.axhline(0, color='red', linestyle='--', alpha=0.5, linewidth=1)
    ax3.axvline(0, color='red', linestyle='--', alpha=0.5, linewidth=1)
    
    # ========================================================================
    # PLOT 4: Lasso Feature Importance
    # ========================================================================
    ax4 = fig.add_subplot(gs[0, 3])
    
    # Get non-zero coefficients
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': np.abs(lasso_model.coef_)
    }).sort_values('coefficient', ascending=False).head(10)
    
    coef_df = coef_df[coef_df['coefficient'] > 0]
    
    bars = ax4.barh(range(len(coef_df)), coef_df['coefficient'], 
                    color='#06A77D', alpha=0.8, edgecolor='black', linewidth=1.5)
    ax4.set_yticks(range(len(coef_df)))
    ax4.set_yticklabels([f.replace('_', ' ').title() for f in coef_df['feature']], 
                        fontsize=9)
    ax4.set_xlabel('|Coefficient|', fontsize=11, fontweight='bold')
    ax4.set_title(f'🎯 Top {len(coef_df)} Lasso Features\n(Out of {len(feature_names)} total)', 
                  fontsize=14, fontweight='bold')
    ax4.grid(axis='x', alpha=0.3, linestyle='--')
    ax4.invert_yaxis()
    
    # ========================================================================
    # PLOT 5: Model Performance Comparison
    # ========================================================================
    ax5 = fig.add_subplot(gs[1, 0:2])
    
    models = ['Ridge', 'Lasso']
    test_mse = [
        validation_summary['ridge']['avg_test_mse'],
        validation_summary['lasso']['avg_test_mse']
    ]
    test_std = [
        validation_summary['ridge']['std_test_mse'],
        validation_summary['lasso']['std_test_mse']
    ]
    
    x = np.arange(len(models))
    bars = ax5.bar(x, test_mse, yerr=test_std, capsize=10,
                   color=['#2E86AB', '#06A77D'], alpha=0.8,
                   edgecolor='black', linewidth=2)
    
    # Add value labels
    for bar, mse, std in zip(bars, test_mse, test_std):
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height + std,
                f'{mse:.6f}',
                ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    ax5.set_xticks(x)
    ax5.set_xticklabels(models, fontsize=12, fontweight='bold')
    ax5.set_ylabel('Test MSE (with std dev)', fontsize=11, fontweight='bold')
    ax5.set_title('🏆 K-Fold Cross-Validation: Model Performance', 
                  fontsize=14, fontweight='bold')
    ax5.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add winner annotation
    winner_idx = 0 if test_mse[0] < test_mse[1] else 1
    ax5.text(winner_idx, test_mse[winner_idx] + test_std[winner_idx] * 2,
            '⭐ WINNER', ha='center', fontsize=12, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='gold', alpha=0.8))
    
    # ========================================================================
    # PLOT 6: Portfolio Metrics Gauge Chart
    # ========================================================================
    ax6 = fig.add_subplot(gs[1, 2:4])
    ax6.axis('off')
    
    # Calculate portfolio metrics
    portfolio_return = sum(
        optimal_weights[asset] * metrics[asset]['return'] 
        for asset in optimal_weights
    )
    portfolio_vol = sum(
        optimal_weights[asset] * metrics[asset]['vol'] 
        for asset in optimal_weights
    )
    portfolio_sharpe = portfolio_return / portfolio_vol if portfolio_vol > 0 else 0
    
    metrics_text = f"""
    📊 PORTFOLIO PERFORMANCE METRICS
    ═══════════════════════════════════════
    
    Expected Return:        {portfolio_return*100:>8.2f}%
    Expected Volatility:    {portfolio_vol*100:>8.2f}%
    Sharpe Ratio:           {portfolio_sharpe:>8.4f}
    
    ═══════════════════════════════════════
    
    🔵 Ridge Regularization:
       • Alpha:            {validation_summary['ridge']['alpha']:>8.4f}
       • Test MSE:         {validation_summary['ridge']['avg_test_mse']:>8.6f}
       • Test R²:          {validation_summary['ridge']['avg_test_r2']:>8.4f}
       • Features Used:    All {len(feature_names)}
    
    🟢 Lasso Regularization:
       • Alpha:            {validation_summary['lasso']['alpha']:>8.6f}
       • Test MSE:         {validation_summary['lasso']['avg_test_mse']:>8.6f}
       • Test R²:          {validation_summary['lasso']['avg_test_r2']:>8.4f}
       • Features Used:    {int(validation_summary['lasso']['avg_features'])}/{len(feature_names)}
    
    ═══════════════════════════════════════
    
    📉 Statistical Test:
       • p-value:          {validation_summary['p_value']:>8.4f}
       • Winner:           {validation_summary['winner'].upper()}
       • Significance:     {"✅ Yes (p < 0.05)" if validation_summary['p_value'] < 0.05 else "❌ No"}
    """
    
    ax6.text(0.1, 0.5, metrics_text, 
             fontfamily='monospace', fontsize=11,
             verticalalignment='center',
             bbox=dict(boxstyle='round', facecolor='lightblue', 
                      alpha=0.3, pad=1.5))
    
    # ========================================================================
    # PLOT 7: Time Series of Portfolio Components
    # ========================================================================
    ax7 = fig.add_subplot(gs[2, :2])
    
    # Plot cumulative returns for each asset
    lookback = min(50, len(features_df))
    recent = features_df.tail(lookback).copy()
    
    for i, asset in enumerate(['BTC', 'ETH', 'DOGE']):
        cumulative = (1 + recent[f'{asset}_return_forecast']).cumprod()
        ax7.plot(range(len(cumulative)), cumulative, 
                label=f'{asset} ({optimal_weights[asset]*100:.1f}%)',
                color=colors[i], linewidth=2.5, alpha=0.8)
    
    # Plot weighted portfolio
    portfolio_returns = sum(
        optimal_weights[asset] * recent[f'{asset}_return_forecast']
        for asset in ['BTC', 'ETH', 'DOGE']
    )
    portfolio_cumulative = (1 + portfolio_returns).cumprod()
    ax7.plot(range(len(portfolio_cumulative)), portfolio_cumulative,
            label='Portfolio (Weighted)', color='black', 
            linewidth=3, linestyle='--', alpha=0.9)
    
    ax7.set_xlabel(f'Days (Last {lookback})', fontsize=11, fontweight='bold')
    ax7.set_ylabel('Cumulative Return', fontsize=11, fontweight='bold')
    ax7.set_title('📈 Cumulative Returns: Assets vs Portfolio', 
                  fontsize=14, fontweight='bold')
    ax7.legend(fontsize=10, loc='best')
    ax7.grid(True, alpha=0.3, linestyle='--')
    ax7.axhline(1, color='red', linestyle='--', alpha=0.5, linewidth=1)
    
    # ========================================================================
    # PLOT 8: Volatility Comparison
    # ========================================================================
    ax8 = fig.add_subplot(gs[2, 2:])
    
    vol_data = {
        'BTC': recent['BTC_volatility_forecast'].mean(),
        'ETH': recent['ETH_volatility_forecast'].mean(),
        'DOGE': recent['DOGE_volatility_forecast'].mean(),
        'Portfolio': portfolio_vol
    }
    
    bars = ax8.bar(vol_data.keys(), [v*100 for v in vol_data.values()],
                   color=colors + ['black'], alpha=0.8,
                   edgecolor='black', linewidth=2)
    
    # Add value labels
    for bar, (name, vol) in zip(bars, vol_data.items()):
        height = bar.get_height()
        ax8.text(bar.get_x() + bar.get_width()/2., height,
                f'{vol*100:.2f}%',
                ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    ax8.set_ylabel('Volatility (%)', fontsize=11, fontweight='bold')
    ax8.set_title('📉 Volatility Comparison\n(Diversification Effect)', 
                  fontsize=14, fontweight='bold')
    ax8.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add diversification benefit annotation
    avg_vol = np.mean([vol_data['BTC'], vol_data['ETH'], vol_data['DOGE']])
    benefit = (avg_vol - vol_data['Portfolio']) / avg_vol * 100
    ax8.text(0.5, 0.95, f'Diversification Benefit: {benefit:.1f}%',
            transform=ax8.transAxes, ha='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    plt.suptitle('Week 5: Portfolio Optimization with Regularization - Comprehensive Analysis', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    plt.show()
    
    print("\n" + "=" * 70)
    print("📊 VISUALIZATION COMPLETE")
    print("=" * 70)
    print("\n✅ All 8 charts successfully generated:")
    print("   1. Portfolio allocation pie chart")
    print("   2. Asset weights breakdown")
    print("   3. Risk-return scatter plot")
    print("   4. Lasso feature importance")
    print("   5. Model performance comparison")
    print("   6. Portfolio metrics summary")
    print("   7. Cumulative returns time series")
    print("   8. Volatility comparison")
    print("=" * 70 + "\n")

In [ ]:
# ============================================================================
# MAIN EXECUTION 
# ============================================================================
def main():
    print("\n" + "🎓" * 35)
    print(" WEEK 5: ML & REGULARIZATION FOR PORTFOLIO OPTIMIZATION")
    print("🎓" * 35 + "\n")
    
    # Step 1: Load Week 4 forecasts
    forecasts = load_week4_forecasts()
    
    # Step 2: Engineer features (prompt-assisted)
    features = engineer_portfolio_features(forecasts)
    
    # Step 3: Prepare ML data
    X, y, feature_names = prepare_ml_data(features)
    
    # Step 4: Ridge Regression (hyperparameter tuning)
    # Note: ridge_scaler not used (only lasso_scaler needed for portfolio optimization)
    ridge_model, _, best_ridge_alpha, _ = fit_ridge_regression(X, y)
    
    # Step 5: Lasso Regression (hyperparameter tuning)
    lasso_model, lasso_scaler, best_lasso_alpha, lasso_n_features = fit_lasso_regression(X, y)
    
    # Step 5A: K-Fold Validation of OPTIMIZED models
    # Note: individual fold results used only within validation function
    validation_summary, _, _ = validate_optimized_models_kfold(
        X, y, best_ridge_alpha, best_lasso_alpha, n_splits=5
    )
    
    # Step 6: Compare models
    # Note: predictions used for internal metrics display only
    _, _ = compare_models(ridge_model, lasso_model, X, y, feature_names)
    
    # Step 7: Portfolio optimization
    optimal_weights = optimize_portfolio_weights(features, lasso_model, lasso_scaler)
    
    # Step 8: Comprehensive visualization
    visualize_portfolio_analysis(optimal_weights, features, lasso_model, 
                                 feature_names, validation_summary)
    
    # Execution verification
    print("\n" + "=" * 70)
    print("🔍 EXECUTION VERIFICATION")
    print("=" * 70)
    print(f"   ✓ Forecasts loaded: {len(forecasts)} days")
    print(f"   ✓ Features engineered: {len(features.columns)} columns")
    print(f"   ✓ ML data prepared: {X.shape[0]} samples, {X.shape[1]} features")
    print(f"   ✓ Ridge alpha: {best_ridge_alpha:.4f}")
    print(f"   ✓ Lasso alpha: {best_lasso_alpha:.6f}")
    print(f"   ✓ Lasso features selected: {lasso_n_features}/{X.shape[1]}")
    print(f"   ✓ K-Fold validation complete: {validation_summary['winner'].upper()} won")
    print(f"   ✓ Portfolio weights optimized:")
    for asset, weight in optimal_weights.items():
        print(f"      • {asset}: {weight*100:.2f}%")
    print("=" * 70)
    
    print("\n" + "=" * 70)
    print("✅ WEEK 5 COMPLETE!")
    print("=" * 70)
    print("\n🎯 Key Takeaways:")
    print("   1. Hyperparameter tuning finds optimal alpha values")
    print("   2. K-Fold CV validates models generalize to unseen data")
    print("   3. Ridge keeps all features, Lasso selects important ones")
    print("   4. Regularization prevents overfitting in portfolio optimization")
    print("   5. Statistical tests confirm model performance differences")
    print("\n📚 Next Week: Tree Ensembles (Random Forest, XGBoost)")
    print("=" * 70 + "\n")
    
    return validation_summary


if __name__ == "__main__":
    results = main()